In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import password
import numpy as np
import matplotlib.pyplot as plt

# Import Data

In [2]:
csv_path = "./csv/perth_market.csv"
perth_df = pd.read_csv(csv_path,encoding="UTF-8")
perth_df.head()

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,-32.193470,115.859553,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN


In [3]:
# Removing unwanted columns
cleaned_perth_df = perth_df.drop(columns = ["NEAREST_SCH_RANK"])


In [4]:
# Convert ADDRESS to FULL ADDRESS which includes the Suburb to get unique adress of sold properties
cleaned_perth_df["ADDRESS"] = cleaned_perth_df[["ADDRESS","SUBURB"]].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)

cleaned_perth_df["ADDRESS"]

0              1 Acorn Place, South Lake
1                     1 Addis Way, Wandi
2               1 Ainsley Court, Camillo
3              1 Albert Street, Bellevue
4                1 Aman Place, Lockridge
                      ...               
33651    9C Gold Street, South Fremantle
33652        9C Pycombe Way, Westminster
33653        9D Pycombe Way, Westminster
33654        9D Shalford Way, Girrawheen
33655        9E Margaret Street, Midland
Name: ADDRESS, Length: 33656, dtype: object

In [5]:
# Display duplicate addresses to check the number of duplicated rows
full_address = cleaned_perth_df["ADDRESS"]
cleaned_perth_df[full_address.isin(full_address[full_address.duplicated()])]

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST
4149,"123 Fairway, Crawley",Crawley,818000,4,1,NaN,160,150,NaN,5500,Daglish Station,3700,02-2018\r,6009,-31.985151,115.815463,SHENTON COLLEGE,3.065177
4150,"123 Fairway, Crawley",Crawley,818000,4,1,NaN,160,150,NaN,5500,Daglish Station,3700,02-2018\r,6009,-31.985152,115.815314,SHENTON COLLEGE,3.059929
4151,"123 Fairway, Crawley",Crawley,818000,4,1,NaN,160,150,NaN,5500,Daglish Station,3700,02-2018\r,6009,-31.985133,115.815176,SHENTON COLLEGE,3.053099
20655,"4 Carmel Road, Carmel",Carmel,955000,4,3,3.0,15761,221,2009.0,21700,Madding,9300,09-2013\r,6076,-32.019395,116.074759,CARMEL ADVENTIST COLLEGE,1.890651
20656,"4 Carmel Road, Carmel",Carmel,955000,4,3,3.0,15761,221,2009.0,21700,Madding,9300,09-2013\r,6076,-32.019657,116.096652,CARMEL ADVENTIST COLLEGE,0.600853


In [6]:
# Drop duplicate data
cleaned_perth_df.drop_duplicates(subset=["ADDRESS"], keep="last", inplace=True)
cleaned_perth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33653 entries, 0 to 33655
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           33653 non-null  object 
 1   SUBURB            33653 non-null  object 
 2   PRICE             33653 non-null  int64  
 3   BEDROOMS          33653 non-null  int64  
 4   BATHROOMS         33653 non-null  int64  
 5   GARAGE            31177 non-null  float64
 6   LAND_AREA         33653 non-null  int64  
 7   FLOOR_AREA        33653 non-null  int64  
 8   BUILD_YEAR        30500 non-null  float64
 9   CBD_DIST          33653 non-null  int64  
 10  NEAREST_STN       33653 non-null  object 
 11  NEAREST_STN_DIST  33653 non-null  int64  
 12  DATE_SOLD         33653 non-null  object 
 13  POSTCODE          33653 non-null  int64  
 14  LATITUDE          33653 non-null  float64
 15  LONGITUDE         33653 non-null  float64
 16  NEAREST_SCH       33653 non-null  object

In [7]:
# Removing properties with land areas over 2500 sqm and price below $100,000
cleaned_perth_df = cleaned_perth_df.loc[(cleaned_perth_df["LAND_AREA"] <= 2500) & (cleaned_perth_df["PRICE"] >= 100000)]
cleaned_perth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30410 entries, 0 to 33655
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           30410 non-null  object 
 1   SUBURB            30410 non-null  object 
 2   PRICE             30410 non-null  int64  
 3   BEDROOMS          30410 non-null  int64  
 4   BATHROOMS         30410 non-null  int64  
 5   GARAGE            28357 non-null  float64
 6   LAND_AREA         30410 non-null  int64  
 7   FLOOR_AREA        30410 non-null  int64  
 8   BUILD_YEAR        27669 non-null  float64
 9   CBD_DIST          30410 non-null  int64  
 10  NEAREST_STN       30410 non-null  object 
 11  NEAREST_STN_DIST  30410 non-null  int64  
 12  DATE_SOLD         30410 non-null  object 
 13  POSTCODE          30410 non-null  int64  
 14  LATITUDE          30410 non-null  float64
 15  LONGITUDE         30410 non-null  float64
 16  NEAREST_SCH       30410 non-null  object

In [8]:
# Convert DATE_SOLD column from MM-YYYY to two columns (Year & Month)
date_splited = cleaned_perth_df["DATE_SOLD"].str.rsplit('-')

year = []
month = []
for date in date_splited:
    year.append(date[1])
    month.append(date[0])

cleaned_perth_df["YEAR_SOLD"] = year
cleaned_perth_df["MONTH_SOLD"] = month

cleaned_perth_df["YEAR_SOLD"] = cleaned_perth_df["YEAR_SOLD"].astype("int64")
cleaned_perth_df["MONTH_SOLD"] = cleaned_perth_df["MONTH_SOLD"].astype("int64")

cleaned_perth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30410 entries, 0 to 33655
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           30410 non-null  object 
 1   SUBURB            30410 non-null  object 
 2   PRICE             30410 non-null  int64  
 3   BEDROOMS          30410 non-null  int64  
 4   BATHROOMS         30410 non-null  int64  
 5   GARAGE            28357 non-null  float64
 6   LAND_AREA         30410 non-null  int64  
 7   FLOOR_AREA        30410 non-null  int64  
 8   BUILD_YEAR        27669 non-null  float64
 9   CBD_DIST          30410 non-null  int64  
 10  NEAREST_STN       30410 non-null  object 
 11  NEAREST_STN_DIST  30410 non-null  int64  
 12  DATE_SOLD         30410 non-null  object 
 13  POSTCODE          30410 non-null  int64  
 14  LATITUDE          30410 non-null  float64
 15  LONGITUDE         30410 non-null  float64
 16  NEAREST_SCH       30410 non-null  object

In [9]:
# Drop NULL values
cleaned_perth_df.dropna(inplace=True)
cleaned_perth_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 25778 entries, 0 to 33654
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           25778 non-null  object 
 1   SUBURB            25778 non-null  object 
 2   PRICE             25778 non-null  int64  
 3   BEDROOMS          25778 non-null  int64  
 4   BATHROOMS         25778 non-null  int64  
 5   GARAGE            25778 non-null  float64
 6   LAND_AREA         25778 non-null  int64  
 7   FLOOR_AREA        25778 non-null  int64  
 8   BUILD_YEAR        25778 non-null  float64
 9   CBD_DIST          25778 non-null  int64  
 10  NEAREST_STN       25778 non-null  object 
 11  NEAREST_STN_DIST  25778 non-null  int64  
 12  DATE_SOLD         25778 non-null  object 
 13  POSTCODE          25778 non-null  int64  
 14  LATITUDE          25778 non-null  float64
 15  LONGITUDE         25778 non-null  float64
 16  NEAREST_SCH       25778 non-null  object

In [10]:
# convert BUILD_YEAR and GARAGE columns from FLOAT to INT
cleaned_perth_df["BUILD_YEAR"] = cleaned_perth_df["BUILD_YEAR"].astype("int64")
cleaned_perth_df["GARAGE"] = cleaned_perth_df["GARAGE"].astype("int64")


In [11]:
# Removing properties which were sold before 2005
cleaned_perth_df = cleaned_perth_df.loc[cleaned_perth_df["YEAR_SOLD"] >= 2005]

In [12]:
# Remove DATE_SOLD column as we now have two seperate columns of the year and month each property were sold 
cleaned_perth_df = cleaned_perth_df.drop(columns = ["DATE_SOLD"])
cleaned_perth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25715 entries, 0 to 33654
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           25715 non-null  object 
 1   SUBURB            25715 non-null  object 
 2   PRICE             25715 non-null  int64  
 3   BEDROOMS          25715 non-null  int64  
 4   BATHROOMS         25715 non-null  int64  
 5   GARAGE            25715 non-null  int64  
 6   LAND_AREA         25715 non-null  int64  
 7   FLOOR_AREA        25715 non-null  int64  
 8   BUILD_YEAR        25715 non-null  int64  
 9   CBD_DIST          25715 non-null  int64  
 10  NEAREST_STN       25715 non-null  object 
 11  NEAREST_STN_DIST  25715 non-null  int64  
 12  POSTCODE          25715 non-null  int64  
 13  LATITUDE          25715 non-null  float64
 14  LONGITUDE         25715 non-null  float64
 15  NEAREST_SCH       25715 non-null  object 
 16  NEAREST_SCH_DIST  25715 non-null  float6

In [13]:
# Remove unusual data
# Remove data where (LAND_AREA<FLOOR_AREA) & (Garage number>10) & (Bathroom number>10) & (Floor Area <50 m2)
cleaned_perth_df = cleaned_perth_df.loc[cleaned_perth_df["LAND_AREA"]>cleaned_perth_df["FLOOR_AREA"]]
cleaned_perth_df = cleaned_perth_df.loc[cleaned_perth_df["GARAGE"]<=10]
cleaned_perth_df = cleaned_perth_df.loc[cleaned_perth_df["BATHROOMS"]<=10]
cleaned_perth_df = cleaned_perth_df.loc[cleaned_perth_df["FLOOR_AREA"]>50]

cleaned_perth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25683 entries, 0 to 33654
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           25683 non-null  object 
 1   SUBURB            25683 non-null  object 
 2   PRICE             25683 non-null  int64  
 3   BEDROOMS          25683 non-null  int64  
 4   BATHROOMS         25683 non-null  int64  
 5   GARAGE            25683 non-null  int64  
 6   LAND_AREA         25683 non-null  int64  
 7   FLOOR_AREA        25683 non-null  int64  
 8   BUILD_YEAR        25683 non-null  int64  
 9   CBD_DIST          25683 non-null  int64  
 10  NEAREST_STN       25683 non-null  object 
 11  NEAREST_STN_DIST  25683 non-null  int64  
 12  POSTCODE          25683 non-null  int64  
 13  LATITUDE          25683 non-null  float64
 14  LONGITUDE         25683 non-null  float64
 15  NEAREST_SCH       25683 non-null  object 
 16  NEAREST_SCH_DIST  25683 non-null  float6

In [14]:
cleaned_perth_df.to_csv("./csv/cleaned_perth_market.csv", index=False)

# Prices in Categorical Format

In [15]:
# # Checking for the right number of bins for PRICE
# min_price = cleaned_perth_df["PRICE"].min()
# max_price = cleaned_perth_df["PRICE"].max()

# bins = np.arange(min_price, (max_price+20001), 50000)
# labels = []
# for i in np.arange(0,len(bins)-1):
#     labels.append(f"{int(bins[i]/1000)}-{int(bins[i+1]/1000)}k")

# labels



In [16]:
# categorical_df = cleaned_perth_df
# categorical_df["PRICE"] = pd.cut(categorical_df["PRICE"], bins=bins, labels=labels, include_lowest=True)

# categorical_df

In [17]:
# categorical_df.info()

In [18]:
# categorical_df.to_csv("./csv/cleaned_price_categorical.csv", index=False)

# Median Price per Suburb

In [20]:
cleaned_perth_df["PRICE_LAND"] = cleaned_perth_df["PRICE"] / cleaned_perth_df["LAND_AREA"]
cleaned_perth_df.head(2)

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,YEAR_SOLD,MONTH_SOLD,PRICE_LAND
0,"1 Acorn Place, South Lake",South Lake,565000,4,2,2,600,160,2003,18300,Cockburn Central Station,1800,6164,-32.11590,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,2018,9,941.666667
1,"1 Addis Way, Wandi",Wandi,365000,3,2,2,351,139,2013,26900,Kwinana Station,4900,6167,-32.19347,115.859553,ATWELL COLLEGE,5.524324,2019,2,1039.886040


In [21]:
avg_price_per_suburb = cleaned_perth_df.groupby("SUBURB").mean()
avg_price_per_suburb.head(2)


,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN_DIST,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH_DIST,YEAR_SOLD,MONTH_SOLD,PRICE_LAND
SUBURB,,,,,,,,,,,,,,,,
Alexander Heights,446691.588785,3.738318,1.822430,2.205607,621.205607,165.598131,1993.289720,13973.831776,7116.822430,6064.0,-31.826918,115.864729,1.921576,2017.710280,7.018692,736.868638
Alfred Cove,844247.368421,3.589474,1.810526,2.021053,580.052632,186.368421,1988.315789,9736.368421,3854.063158,6154.0,-32.033908,115.816117,1.487421,2016.578947,7.368421,1605.300498


In [20]:
avg_price_per_suburb["PRICE_LAND"].to_csv("./csv/avg_price_suburb.csv")
